In [ ]:
!nvidia-smi

Fri Jul 22 00:22:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install git+https://github.com/huggingface/transformers datasets
!pip list | grep -E 'transformers|tokenizers'

In [ ]:
from datasets import load_dataset
#dataset = load_dataset("txt", data_files="/content/final_filtered.txt")

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
%env WANDB_PROJECT = tajberto
%env WANDB_WATCH = all
%env WANDB_LOG_MODEL = true

In [ ]:
#!gzip -d /content/tg.txt.gz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil 
shutil.copy("/content/drive/MyDrive/merged_data_preprocessed.txt", "/content/final_filtered.txt") 
 
#final_filtered_new.txt", 

'/content/final_filtered.txt'

In [ ]:
%%time 
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

path = "/content/final_filtered.txt"

# Customize training
tokenizer.train(files=path, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 365 ms, sys: 218 ms, total: 582 ms
Wall time: 273 ms


Now let's save files to disk

In [ ]:
#!wget https://huggingface.co/roberta-base/raw/main/config.json -P /content/TajBERTo

In [ ]:
mkdir /content/BERT-domain

In [ ]:
tokenizer.save_model("/content/BERT-domain")

['/content/BERT-domain/vocab.json', '/content/BERT-domain/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./BERT-domain/vocab.json",
    "./BERT-domain/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Hello")

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode("Hello").tokens

['<s>', 'H', 'el', 'lo', '</s>']

In [ ]:
from tokenizers.decoders import ByteLevel
decoder = ByteLevel()
decoder.decode(['<s>', 'H', 'el', 'lo', '</s>'])

'<s>Hello</s>'

## 3. Train a language model from scratch


> We’ll train a RoBERTa-like model, which is a BERT-like with a couple of changes (check the [documentation](https://huggingface.co/transformers/model_doc/roberta.html) for more details).

As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.


In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

### We'll define the following config for the model

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("google/bert_uncased_L-2_H-128_A-2")

In [ ]:
config

BertConfig {
  "_name_or_path": "google/bert_uncased_L-2_H-128_A-2",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Now let's re-create our tokenizer in transformers

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("/content/RoBERTa-tg", max_len=512)

In [ ]:
from transformers import AutoTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("/content/Bert", max_len=512)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 84 million parameters

83504416

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path= "/content/final_filtered.txt", #"/content/tg.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Finally, we are all set to initialize our Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/RoBERTa-tg",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=128,#128 next time
    save_steps=10_000, #500 better
    save_total_limit=1,
    prediction_loss_only=True,
    seed=32, report_to="wandb", run_name="roberta-base 128 batch", 
)

trainer = Trainer(
    model= model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

### Start training

#### 🎉 Save final model (+ tokenizer + config) to disk

In [ ]:
trainer.save_model()

Saving model checkpoint to /content/RoBERTa-tg
Configuration saved in /content/RoBERTa-tg/config.json
Model weights saved in /content/RoBERTa-tg/pytorch_model.bin


In [ ]:
wandb.finish()

train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,16825


In [ ]:
trainer.push_to_hub()

/content/RoBERTa-tg is already a clone of https://huggingface.co/muhtasham/RoBERTa-tg. Make sure you pull the latest changes with `repo.git_pull()`.
Saving model checkpoint to /content/RoBERTa-tg
Configuration saved in /content/RoBERTa-tg/config.json
Model weights saved in /content/RoBERTa-tg/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.34k/319M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.17k/3.17k [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/muhtasham/RoBERTa-tg
   27ae1f8..04c4daf  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/muhtasham/RoBERTa-tg
   04c4daf..643187d  main -> main



'https://huggingface.co/muhtasham/RoBERTa-tg/commit/04c4daf0bab5479b6a64cd197acb9da6b165e838'

## 4. Check that the LM actually trained

Aside from looking at the training and eval losses going down, the easiest way to check whether our language model is learning anything interesting is via the `FillMaskPipeline`.

Pipelines are simple wrappers around tokenizers and models, and the 'fill-mask' one will let you input a sequence containing a masked token (here, `<mask>`) and return a list of the most probable filled sequences, with their probabilities.



In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model = model,
    tokenizer = tokenizer,
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("muhtasham/TajBERTo")

model = AutoModelForMaskedLM.from_pretrained("muhtasham/TajBERTo")

In [ ]:
# The sun <mask>.
# =>

#fill_mask("Аз Гулобод то маркази ҷамоат <mask>.")
#unedited dataset

In [ ]:
fill_mask("<mask> ба ин сайти шумо медароям.")

In [ ]:
fill_mask("Пойтахти <mask> Душанбе")

In [ ]:
fill_mask("Абдукаримбий бародари <mask> Абдураҳимбий буд.")

In [ ]:
fill_mask("Аз ин рӯ <mask> Боми Ҷаҳон  меноманд")

In [ ]:
fill_mask("Салом <mask> ")

In [ ]:
fill_mask("Алейкум <mask> ")

In [ ]:
fill_mask("Номи ман <mask>")

In [ ]:
import shutil
shutil.copytree("/content/RoBERTa-tg","/content/drive/MyDrive/raw/RoBERTa-tg")

'/content/drive/MyDrive/raw/RoBERTa-tg'